In [141]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    data['minute'] = data['context_timestamp_tmp'].dt.minute
    data['halfhour'] = (data['hour']+1)*3+(data['minute']/20).astype('int')
    del data['context_timestamp_tmp']
    del data['minute']
    del data['week']
    del data['hour']
    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']
#     del data['shop_score_service']
#     del data['shop_score_description']
#     del data['shop_review_positive_rate']
#     del data['shop_score_delivery']
    return data

In [233]:
print('train')
train = pd.read_csv('train.csv',index_col=False)
dataa = pd.read_csv('train_all.csv',index_col=False)

train


## 特征选择

In [234]:
dataa.columns

Index(['instance_id', 'item_category_list_num', 'item_property_list_num',
       'hour_num', 'item_click', 'shop_score_service_num',
       'shop_score_description_num', 'shop_review_positive_rate_num',
       'shop_score_delivery_num', 'user_click', 'property_0_click',
       'property_1_click', 'property_2_click', 'shop_click', 'brand_click',
       'category_1_pvnum', 'property_0_pvnum', 'property_1_pvnum',
       'property_2_pvnum', 'item_id_pvnum', 'occupation_click'],
      dtype='object')

In [235]:
tezheng = ['instance_id','item_id_pvnum', 'occupation_click','category_1_pvnum']

In [236]:
dataa = dataa[tezheng]

In [237]:
train = pd.merge(train,dataa,on=['instance_id'],how='left')
train = pre_process(train)

预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [238]:
all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

# train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)


all_shape (478087, 34)
2018-09-24 23:59:47
(357066, 34)
(121021, 34)


In [239]:

print('test')
test_a = pd.read_csv('test.csv',index_col=False)
data2a = pd.read_csv('test_all.csv',index_col=False) 

# 0.83108
# del train[['user_id', 'category_1', 'property_2', 'property_0']]
# del val[['user_id', 'category_1', 'property_2', 'property_0']]
# del test_a[['user_id', 'category_1', 'property_2', 'property_0']]
# 这里是增加的内容


test


## 特征选择

In [240]:
data2a = data2a[tezheng]

In [241]:
test_a.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [242]:
test_a = pd.merge(test_a,data2a,on=['instance_id'],how='left')
print(test_a.shape)
test_a = pre_process(test_a)
del train['item_city_id']
del val['item_city_id']
del test_a['item_city_id']
del train['shop_id']
del val['shop_id']
del test_a['shop_id']

(18371, 29)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [243]:

import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count




In [244]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)
# print(train.columns)
train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)

# 删掉上面增加的用户点击量与商品点击量
# del train['user_count']
# del train['item_count']
# del val['user_count']
# del val['item_count']
# del test_a['user_count']
# del test_a['item_count']

# user_item_count = pd.read_csv


y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')

print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data


2018-09-18 00:00:01
2018-09-16 00:00:01
NaT
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43
(18371, 32)


In [245]:

print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
# print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
# feat_set = list(test_a.columns)

del train['user_id']
del test_a['user_id']
del val['user_id']
# 8288

print(train.columns)

# 自己使用onehot编码
train['x_train'] = 1
test_a['x_test'] = 1
val['x_val'] = 1

all_data = pd.concat([train,test_a,val])
all_data = pd.get_dummies(all_data)
print(all_data.shape)

X_train = all_data.loc[all_data['x_train'].notnull()]
X_test = all_data.loc[all_data['x_test'].notnull()]
X_val = all_data.loc[all_data['x_val'].notnull()]

del X_train['x_train']
del X_train['x_test']
del X_train['x_val']
del X_test['x_train']
del X_test['x_test']
del X_test['x_val']
del X_val['x_train']
del X_val['x_test']
del X_val['x_val']


baseline ing ... ...
Index(['item_id', 'item_brand_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_page_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'item_id_pvnum', 'occupation_click',
       'category_1_pvnum', 'category_0', 'category_1', 'category_2',
       'property_0', 'property_1', 'property_2', 'halfhour', 'user_count',
       'item_count'],
      dtype='object')
(496458, 585)


In [246]:
# LogisticRegression?

In [247]:

# lr = LogisticRegression(C=5.0,penalty='l1')

# lr.fit(X_train, y_train)


In [248]:

# proba_val = lr.predict_proba(X_val)[:,1]
# proba_sub = lr.predict_proba(X_test)[:,1]
# print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
# print(log_loss(y_val,proba_val))
# print()

In [263]:

import lightgbm as lgb
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=50,
#                         max_depth = 8,
                        
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.7,
                        reg_alpha = 2
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)



[1]	valid_0's binary_logloss: 0.683897
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674838
[3]	valid_0's binary_logloss: 0.665946
[4]	valid_0's binary_logloss: 0.657232
[5]	valid_0's binary_logloss: 0.648686
[6]	valid_0's binary_logloss: 0.640306
[7]	valid_0's binary_logloss: 0.632079
[8]	valid_0's binary_logloss: 0.624014
[9]	valid_0's binary_logloss: 0.616094
[10]	valid_0's binary_logloss: 0.608318
[11]	valid_0's binary_logloss: 0.600686
[12]	valid_0's binary_logloss: 0.593197
[13]	valid_0's binary_logloss: 0.585845
[14]	valid_0's binary_logloss: 0.578627
[15]	valid_0's binary_logloss: 0.571532
[16]	valid_0's binary_logloss: 0.56456
[17]	valid_0's binary_logloss: 0.557709
[18]	valid_0's binary_logloss: 0.550978
[19]	valid_0's binary_logloss: 0.54437
[20]	valid_0's binary_logloss: 0.537872
[21]	valid_0's binary_logloss: 0.531485
[22]	valid_0's binary_logloss: 0.525199
[23]	valid_0's binary_logloss: 0.519024
[24]	valid_0's binary_loglos

[207]	valid_0's binary_logloss: 0.123627
[208]	valid_0's binary_logloss: 0.123105
[209]	valid_0's binary_logloss: 0.12259
[210]	valid_0's binary_logloss: 0.122077
[211]	valid_0's binary_logloss: 0.121571
[212]	valid_0's binary_logloss: 0.121076
[213]	valid_0's binary_logloss: 0.120583
[214]	valid_0's binary_logloss: 0.120096
[215]	valid_0's binary_logloss: 0.119615
[216]	valid_0's binary_logloss: 0.11914
[217]	valid_0's binary_logloss: 0.118673
[218]	valid_0's binary_logloss: 0.118215
[219]	valid_0's binary_logloss: 0.117757
[220]	valid_0's binary_logloss: 0.117306
[221]	valid_0's binary_logloss: 0.11686
[222]	valid_0's binary_logloss: 0.11642
[223]	valid_0's binary_logloss: 0.115984
[224]	valid_0's binary_logloss: 0.115557
[225]	valid_0's binary_logloss: 0.115132
[226]	valid_0's binary_logloss: 0.114713
[227]	valid_0's binary_logloss: 0.114302
[228]	valid_0's binary_logloss: 0.113895
[229]	valid_0's binary_logloss: 0.113491
[230]	valid_0's binary_logloss: 0.113093
[231]	valid_0's bina

[409]	valid_0's binary_logloss: 0.0854018
[410]	valid_0's binary_logloss: 0.085361
[411]	valid_0's binary_logloss: 0.0853235
[412]	valid_0's binary_logloss: 0.0852887
[413]	valid_0's binary_logloss: 0.0852524
[414]	valid_0's binary_logloss: 0.0852179
[415]	valid_0's binary_logloss: 0.085181
[416]	valid_0's binary_logloss: 0.0851441
[417]	valid_0's binary_logloss: 0.0851095
[418]	valid_0's binary_logloss: 0.0850737
[419]	valid_0's binary_logloss: 0.0850401
[420]	valid_0's binary_logloss: 0.085007
[421]	valid_0's binary_logloss: 0.084975
[422]	valid_0's binary_logloss: 0.0849442
[423]	valid_0's binary_logloss: 0.0849103
[424]	valid_0's binary_logloss: 0.0848779
[425]	valid_0's binary_logloss: 0.0848463
[426]	valid_0's binary_logloss: 0.0848147
[427]	valid_0's binary_logloss: 0.0847858
[428]	valid_0's binary_logloss: 0.084758
[429]	valid_0's binary_logloss: 0.0847307
[430]	valid_0's binary_logloss: 0.0846989
[431]	valid_0's binary_logloss: 0.0846705
[432]	valid_0's binary_logloss: 0.08464

[608]	valid_0's binary_logloss: 0.0825296
[609]	valid_0's binary_logloss: 0.0825251
[610]	valid_0's binary_logloss: 0.0825223
[611]	valid_0's binary_logloss: 0.0825176
[612]	valid_0's binary_logloss: 0.0825135
[613]	valid_0's binary_logloss: 0.0825104
[614]	valid_0's binary_logloss: 0.0825086
[615]	valid_0's binary_logloss: 0.0825056
[616]	valid_0's binary_logloss: 0.0825014
[617]	valid_0's binary_logloss: 0.0824996
[618]	valid_0's binary_logloss: 0.0824941
[619]	valid_0's binary_logloss: 0.0824904
[620]	valid_0's binary_logloss: 0.0824876
[621]	valid_0's binary_logloss: 0.0824823
[622]	valid_0's binary_logloss: 0.0824786
[623]	valid_0's binary_logloss: 0.0824754
[624]	valid_0's binary_logloss: 0.0824722
[625]	valid_0's binary_logloss: 0.0824704
[626]	valid_0's binary_logloss: 0.0824694
[627]	valid_0's binary_logloss: 0.0824667
[628]	valid_0's binary_logloss: 0.0824636
[629]	valid_0's binary_logloss: 0.0824611
[630]	valid_0's binary_logloss: 0.0824592
[631]	valid_0's binary_logloss: 0.

[808]	valid_0's binary_logloss: 0.082134
[809]	valid_0's binary_logloss: 0.0821337
[810]	valid_0's binary_logloss: 0.0821348
[811]	valid_0's binary_logloss: 0.0821331
[812]	valid_0's binary_logloss: 0.0821337
[813]	valid_0's binary_logloss: 0.0821334
[814]	valid_0's binary_logloss: 0.0821331
[815]	valid_0's binary_logloss: 0.0821331
[816]	valid_0's binary_logloss: 0.0821327
[817]	valid_0's binary_logloss: 0.0821328
[818]	valid_0's binary_logloss: 0.0821332
[819]	valid_0's binary_logloss: 0.0821325
[820]	valid_0's binary_logloss: 0.082131
[821]	valid_0's binary_logloss: 0.0821325
[822]	valid_0's binary_logloss: 0.0821328
[823]	valid_0's binary_logloss: 0.0821327
[824]	valid_0's binary_logloss: 0.0821328
[825]	valid_0's binary_logloss: 0.0821312
[826]	valid_0's binary_logloss: 0.0821296
[827]	valid_0's binary_logloss: 0.0821302
[828]	valid_0's binary_logloss: 0.0821289
[829]	valid_0's binary_logloss: 0.082129
[830]	valid_0's binary_logloss: 0.0821296
[831]	valid_0's binary_logloss: 0.082

[1008]	valid_0's binary_logloss: 0.0820615
[1009]	valid_0's binary_logloss: 0.0820614
[1010]	valid_0's binary_logloss: 0.082061
[1011]	valid_0's binary_logloss: 0.0820609
[1012]	valid_0's binary_logloss: 0.0820602
[1013]	valid_0's binary_logloss: 0.08206
[1014]	valid_0's binary_logloss: 0.0820598
[1015]	valid_0's binary_logloss: 0.0820588
[1016]	valid_0's binary_logloss: 0.0820582
[1017]	valid_0's binary_logloss: 0.0820579
[1018]	valid_0's binary_logloss: 0.0820583
[1019]	valid_0's binary_logloss: 0.0820577
[1020]	valid_0's binary_logloss: 0.0820566
[1021]	valid_0's binary_logloss: 0.0820578
[1022]	valid_0's binary_logloss: 0.0820564
[1023]	valid_0's binary_logloss: 0.0820563
[1024]	valid_0's binary_logloss: 0.0820565
[1025]	valid_0's binary_logloss: 0.0820572
[1026]	valid_0's binary_logloss: 0.0820574
[1027]	valid_0's binary_logloss: 0.0820564
[1028]	valid_0's binary_logloss: 0.0820544
[1029]	valid_0's binary_logloss: 0.0820538
[1030]	valid_0's binary_logloss: 0.0820541
[1031]	valid_0

[1200]	valid_0's binary_logloss: 0.0820416
[1201]	valid_0's binary_logloss: 0.0820419
[1202]	valid_0's binary_logloss: 0.0820426
[1203]	valid_0's binary_logloss: 0.0820441
[1204]	valid_0's binary_logloss: 0.082044
[1205]	valid_0's binary_logloss: 0.0820446
[1206]	valid_0's binary_logloss: 0.0820447
[1207]	valid_0's binary_logloss: 0.0820462
[1208]	valid_0's binary_logloss: 0.0820456
[1209]	valid_0's binary_logloss: 0.0820446
[1210]	valid_0's binary_logloss: 0.0820449
[1211]	valid_0's binary_logloss: 0.0820434
[1212]	valid_0's binary_logloss: 0.0820427
[1213]	valid_0's binary_logloss: 0.0820425
[1214]	valid_0's binary_logloss: 0.082042
[1215]	valid_0's binary_logloss: 0.0820414
[1216]	valid_0's binary_logloss: 0.0820407
[1217]	valid_0's binary_logloss: 0.0820391
[1218]	valid_0's binary_logloss: 0.0820388
[1219]	valid_0's binary_logloss: 0.0820382
[1220]	valid_0's binary_logloss: 0.0820388
[1221]	valid_0's binary_logloss: 0.0820386
[1222]	valid_0's binary_logloss: 0.0820393
[1223]	valid_

[1398]	valid_0's binary_logloss: 0.0820509
[1399]	valid_0's binary_logloss: 0.0820499
[1400]	valid_0's binary_logloss: 0.0820508
[1401]	valid_0's binary_logloss: 0.0820506
[1402]	valid_0's binary_logloss: 0.0820495
[1403]	valid_0's binary_logloss: 0.0820494
[1404]	valid_0's binary_logloss: 0.0820511
[1405]	valid_0's binary_logloss: 0.082049
[1406]	valid_0's binary_logloss: 0.0820497
[1407]	valid_0's binary_logloss: 0.0820492
[1408]	valid_0's binary_logloss: 0.0820507
[1409]	valid_0's binary_logloss: 0.0820514
[1410]	valid_0's binary_logloss: 0.082052
[1411]	valid_0's binary_logloss: 0.0820512
[1412]	valid_0's binary_logloss: 0.0820501
[1413]	valid_0's binary_logloss: 0.0820502
[1414]	valid_0's binary_logloss: 0.0820503
[1415]	valid_0's binary_logloss: 0.0820501
[1416]	valid_0's binary_logloss: 0.08205
[1417]	valid_0's binary_logloss: 0.0820514
[1418]	valid_0's binary_logloss: 0.0820525
[1419]	valid_0's binary_logloss: 0.0820518
[1420]	valid_0's binary_logloss: 0.0820521
[1421]	valid_0'

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=50, objective='binary', random_state=None,
       reg_alpha=2, reg_lambda=0.0, silent=True, subsample=0.7,
       subsample_for_bin=200000, subsample_freq=1)

In [264]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

# print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0816995451384

Start predicting...
0.0811428783673
0.0820321287875


In [265]:

# import numpy as np
# bes = 1
# for i in np.arange(0,1,0.01):
#     if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
#         bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
# print(bes)#0.083131236937

# xx_analyse = pd.DataFrame()
# xx_analyse['ture'] = list(y_val)
# xx_analyse['pre'] = list(proba_val)
# xx_analyse['pre_1'] = list(y_pred_1)
# xx_analyse.to_csv('temp.csv',index=False)
#
# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)
# sub['predicted_score'] = list(y_sub_1)
# sub.to_csv('feather/20180327.txt',sep=" ",index=False)

In [266]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180414.txt',sep=" ",index=False)